In [ ]:
using Revise

In [ ]:
using Optim
using ForwardDiff
using Plots; gr()
using ColorTypes
using ReverseDiff
using Base.Test
using BenchmarkTools

In [ ]:
import Nets
import LearningMPC

In [ ]:
target = x -> [sin(x[1])]
# target = x -> x

In [ ]:
training_pts = linspace(-2π, 0, 5)
sample(x) = ([x], hcat(target(x), ForwardDiff.jacobian(target, [x])))
training_data = [sample(x) for x in training_pts];
# training_data = Tuple{Vector{Float64}, Matrix{Float64}}[]

In [ ]:
net = Nets.Net(zeros(Nets.Params{Float64}, [1, 10, 10, 1]), Nets.elu)
for I in eachindex(net.params.data)
    net.params.data[I] += 0.1 * randn()
end

f, g! = Nets.cost_function(net, training_data, 0.5)
@inferred f(net.params.data)
@inferred g!(similar(net.params.data), net.params.data)
@inferred Nets.predict_sensitivity(net, zeros(1))
∇ = similar(net.params.data);
# @benchmark $g!($∇, $(net.params.data))

In [ ]:
@show f(net.params.data)
solver = LBFGS()
options = Optim.Options()
@time results = optimize(f, g!, copy(net.params.data), solver, options)
net.params.data .= results.minimizer;
@show f(net.params.data)

for i in 1:10
    push!(training_data, sample(randn() + 0.5))
end
@show f(net.params.data)

@time results = optimize(f, g!, copy(net.params.data), solver, options)
net.params.data .= results.minimizer;
@show f(net.params.data)

results

In [ ]:
plt = plot([x[1] for (x, yJ) in training_data], 
    [yJ[1] for (x, yJ) in training_data],
    line=nothing,
    marker=:dot)
for (x, yJ) in training_data
    y = yJ[1, 1]
    slope = yJ[1, 2]
    θ = atan(slope)
    δx = 0.1 * cos(θ)
    xs = [x[1] - δx, x[1] + δx]
    ys = [y - slope * δx, y + slope * δx]
    plot!(plt, xs, ys, color=colorant"red", legend=nothing)
end

x_samples = linspace(-2π, 2π, 101)
plot!(plt, x_samples, [net([x])[1] for x in x_samples], color=colorant"blue")

plt